# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Список категорий
['Ювелирные изделия', 'Ремонт', 'Отели', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Алименты', 'Магазины косметики и Салоны красоты', 'Аптеки']
В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями

Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 14 признаков, то есть семь категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1kYYRWFmQSeE_MfRUOO1JjrU5uskf9Co3"})   # замените id на id своего файла
downloaded.GetContentFile('transactions.csv')
transactions = pd.read_csv('transactions.csv',   nrows=1000000)


downloaded = drive.CreateFile({'id':"1csB_Jb20q7Cn3pF-4ZH_R1NOE9fhswyZ"})   # замените id на id своего файла
downloaded.GetContentFile('gender_train.csv')
gender_train = pd.read_csv('gender_train.csv')

downloaded = drive.CreateFile({'id':"16g6aSiirBY-CnrvDwPEs39TFH3UTSyjT"})   # замените id на id своего файла
downloaded.GetContentFile('tr_mcc_codes.csv')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';')

downloaded = drive.CreateFile({'id':"1K-3_yIdT0viRygRx1E1e3b1dGw8w67-j"})   # замените id на id своего файла
downloaded.GetContentFile('tr_types.csv')
tr_types = pd.read_csv('tr_types.csv', sep=";")

In [ ]:
transactions.sample(n=10)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
494085,40133287,414 15:39:52,5411,1010,-4417.72,067864
734033,39351532,390 16:59:59,5411,1010,-25828.03,10179467
387063,70780820,108 09:59:23,6010,7030,6558.07,NaN
552845,62030642,318 13:31:01,6011,2010,-26950.99,650211
746942,701335,357 18:10:59,4829,4071,-224.59,888901
837860,67818644,128 16:37:25,6011,2011,-33688.74,NaN
113915,8928713,192 17:51:39,6011,2010,-89836.63,NaN
36390,55045775,63 00:00:00,7011,1110,-79550.34,NaN
937875,2208164,124 14:26:01,4829,2370,-6531.80,NaN
45357,84027814,292 16:31:18,8999,1010,-1122.96,10166202


In [ ]:
gender_train.sample(n=5)

,customer_id,gender
5566,4466684,1
6358,99787257,0
6613,22717722,0
1032,48413853,0
4356,82615859,1


In [ ]:
gender_train.isna().sum()

customer_id    0
gender         0
dtype: int64

In [ ]:
my_list = feature_list[::-2]


['ювелирные изделия',
 'ремонт',
 'отели',
 'автозапчасти, автошины и станции техобслуживания (а также сто)',
 'алименты',
 'магазины косметики и салоны красоты',
 'аптеки']

In [ ]:
tr_mcc_codes.sample(n=10)

,mcc_code,mcc_description
106,5949,"Магазины ткани, ниток рукоделия, шитья"
37,5122,"Лекарства, их распространители, аптеки"
52,5310,"Магазины, торгующие по сниженным ценам"
12,4214,"Агентства по автотранспортным перевозкам, мест..."
168,8043,"Оптика, оптические товары и очки"
41,5172,Нефть и нефтепродукты
140,7311,Рекламные услуги
176,8641,"Ассоциации — гражданские, социальные и братские"
121,5995,Зоомагазины
123,6010,Финансовые институты — снятие наличности вручную


In [ ]:
import re
jevelery_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('ювелир', flags=re.IGNORECASE) ]['mcc_code']
fix_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('ремонт|почин|восстанов', flags=re.IGNORECASE)]['mcc_code']
hotel_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('отел|хостел|мотел', flags=re.IGNORECASE)]['mcc_code']
car_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('автозапчаст|автошин|шин|сто |техобслуж', flags=re.IGNORECASE)]['mcc_code']
tax_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('алимент', flags=re.IGNORECASE)]['mcc_code']
biauty_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('космет|красот|парикхм|парфюм', flags=re.IGNORECASE)]['mcc_code']
farm = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('аптек|фарма', flags=re.IGNORECASE)]['mcc_code']


In [ ]:
transactions = pd.merge(transactions[['amount','customer_id','mcc_code']], 
                        tr_mcc_codes.drop(columns=['mcc_description']), how='left')



In [ ]:
transactions.sample(n=10)

,amount,customer_id,mcc_code
899853,-1122.96,117556,4814
737709,-13003.85,43394138,5812
587275,-2463.77,61853422,5912
807132,-7860.71,87069325,4814
843679,-2245.92,74630945,4814
114185,-20213.24,8928713,6011
760650,-2245.92,54186907,4814
291823,-51656.06,76282911,6011
43814,-1122.96,2314154,4814
356251,-26647.79,17992634,5541


In [ ]:
transactions.shape

(570814, 4)

In [ ]:
income = transactions[transactions['amount'] > 0]
outcome = transactions[transactions['amount'] < 0]

In [ ]:
income.head()

,amount,customer_id,mcc_code
1,56147.89,39026145,6011
16,224591.58,39026145,6010
42,22459.16,39026145,6011
67,22459.16,39026145,6011
71,96574.38,39026145,6010


In [ ]:
out_fix = outcome[outcome['mcc_code'].isin(fix_code.values)].drop(columns=['mcc_code'])
out_fix = out_fix.groupby('customer_id').agg(
    amount_out_fix=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()

out_jevelery = outcome[outcome['mcc_code'].isin(jevelery_code.values)].drop(columns=['mcc_code'])
out_jevelery = out_jevelery.groupby('customer_id').agg(
    amount_out_jev=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()

out_hotel = outcome[outcome['mcc_code'].isin(hotel_code.values)].drop(columns=['mcc_code'])
out_hotel = out_hotel.groupby('customer_id').agg(
    amount_out_hotel=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()
out_car = outcome[outcome['mcc_code'].isin(car_code.values)].drop(columns=['mcc_code'])
out_car = out_car.groupby('customer_id').agg(
    amount_out_car=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()

out_tax = outcome[outcome['mcc_code'].isin(tax_code.values)].drop(columns=['mcc_code'])
out_tax = out_tax.groupby('customer_id').agg(
    amount_out_tax=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()
out_biauty = outcome[outcome['mcc_code'].isin(biauty_code.values)].drop(columns=['mcc_code'])
out_biauty = out_biauty.groupby('customer_id').agg(
    amount_out_b=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()
out_farm = outcome[outcome['mcc_code'].isin(farm.values)].drop(columns=['mcc_code'])
out_farm = out_farm.groupby('customer_id').agg(
    amount_out_farm=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()


In [ ]:
in_fix = income[income['mcc_code'].isin(fix_code.values)].drop(columns=['mcc_code'])
in_fix = in_fix.groupby('customer_id').agg(
    amount_in_fix=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()

in_jevelery = income[income['mcc_code'].isin(jevelery_code.values)].drop(columns=['mcc_code'])
in_jevelery = in_jevelery.groupby('customer_id').agg(
    amount_in_jev=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()

in_hotel = income[income['mcc_code'].isin(hotel_code.values)].drop(columns=['mcc_code'])
in_hotel = in_hotel.groupby('customer_id').agg(
    amount_in_hotel=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()
in_car = income[income['mcc_code'].isin(car_code.values)].drop(columns=['mcc_code'])
in_car = in_car.groupby('customer_id').agg(
    amount_in_car=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()

in_tax = income[income['mcc_code'].isin(tax_code.values)].drop(columns=['mcc_code'])
in_tax = in_tax.groupby('customer_id').agg(
    amount_in_tax=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()
in_biauty = income[income['mcc_code'].isin(biauty_code.values)].drop(columns=['mcc_code'])
in_biauty = in_biauty.groupby('customer_id').agg(
    amount_in_b=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()
in_farm = income[income['mcc_code'].isin(farm.values)].drop(columns=['mcc_code'])
in_farm = in_farm.groupby('customer_id').agg(
    amount_in_farm=pd.NamedAgg(column='amount', aggfunc = sum)
).reset_index()


In [ ]:
result = pd.merge(in_fix, in_jevelery, on=['customer_id'], how='outer')
result = pd.merge(result,in_hotel , on=['customer_id'], how='outer')
result = pd.merge(result,in_car , on=['customer_id'], how='outer')
result = pd.merge(result,in_tax , on=['customer_id'], how='outer')
result = pd.merge(result,in_biauty , on=['customer_id'], how='outer')
result = pd.merge(result,in_farm , on=['customer_id'], how='outer')


In [ ]:

result = pd.merge(result, out_fix , on=['customer_id'], how='outer')
result = pd.merge(result, out_jevelery , on=['customer_id'], how='outer')
result = pd.merge(result, out_hotel , on=['customer_id'], how='outer')
result = pd.merge(result, out_car , on=['customer_id'], how='outer')
result = pd.merge(result, out_tax , on=['customer_id'], how='outer')
result = pd.merge(result, out_biauty , on=['customer_id'], how='outer')
result = pd.merge(result, out_farm , on=['customer_id'], how='outer')



In [ ]:
result = result.reset_index()

In [ ]:
result= pd.merge(result, gender_train,on=['customer_id'], how='inner')

result.drop(columns=['customer_id'])

,amount_in_fix,amount_in_jev,amount_in_hotel,amount_in_car,amount_in_tax,amount_in_b,amount_in_farm,amount_out_fix,amount_out_jev,amount_out_hotel,amount_out_car,amount_out_tax,amount_out_b,amount_out_farm,gender
0,224591.58,NaN,NaN,NaN,NaN,NaN,NaN,-815689.21,-190363.82,NaN,-648050.93,NaN,-160516.72,-311739.87,1
1,4828.72,NaN,NaN,NaN,NaN,NaN,NaN,-96574.38,NaN,NaN,-198726.94,NaN,-6510.91,-51739.16,1
2,NaN,43346.17,NaN,NaN,NaN,NaN,NaN,NaN,-1291199.88,NaN,-16816.97,NaN,-645027.03,-637004.15,0
3,NaN,NaN,281334.19,NaN,NaN,NaN,NaN,-2530450.84,-1535936.88,-10644156.21,-434728.44,NaN,-2874868.10,-1147619.16,0
4,NaN,NaN,1297690.14,NaN,NaN,NaN,NaN,-5645851.34,-34504955.81,-35367040.28,-293284.48,NaN,-1869364.67,-2382933.25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-149584.50,1
884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1942.72,0
885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38066.03,0
886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-77023.68,1


In [ ]:
result = result.fillna(0).drop(columns=['customer_id'])

In [ ]:
X = result.drop(columns=['gender'])
Y = result['gender']

In [ ]:
d = X.max() - X.min()
d['amount_in_tax'] = 1
d['amount_out_tax'] = 1

In [ ]:
X = (X - X.mean()) /d

# Задание 2

Модель - многослойная нейронная сеть с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Нужно отдельно перечислить все гиперпараметры и их значения вот в таком формате (значения приведены для примера, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация: ReLU
* и т.д.

Настроить параметры модели.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
import keras

In [ ]:
import keras

from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
y_train=keras.utils.to_categorical(y_train,2)
y_test2 = y_test 
y_test=keras.utils.to_categorical(y_test,2)

In [ ]:
model1 = Sequential([
                     Dense(256, activation='relu'),
                     Dropout(0.5),
                     Dense(units = 128, activation = 'relu'),
                     Dense(units = 64, activation='relu'),
                     Dropout(0.2),
                     Dense(units = 2, activation='softmax')])

model1.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [ ]:
history1 = model1.fit(X_train, y_train,
                      epochs=50,
                      batch_size = 20,
                      validation_data=(X_test,y_test))

Epoch 1/50
32/32 [==============================] - 1s 11ms/step - loss: 0.6836 - accuracy: 0.5960 - val_loss: 0.6447 - val_accuracy: 0.6667
Epoch 2/50
32/32 [==============================] - 0s 4ms/step - loss: 0.6209 - accuracy: 0.6885 - val_loss: 0.6034 - val_accuracy: 0.6779
Epoch 3/50
32/32 [==============================] - 0s 4ms/step - loss: 0.5873 - accuracy: 0.6697 - val_loss: 0.6025 - val_accuracy: 0.6779
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.5623 - accuracy: 0.6980 - val_loss: 0.5983 - val_accuracy: 0.6816
Epoch 5/50
32/32 [==============================] - 0s 4ms/step - loss: 0.5599 - accuracy: 0.7091 - val_loss: 0.5839 - val_accuracy: 0.7041
Epoch 6/50
32/32 [==============================] - 0s 5ms/step - loss: 0.5504 - accuracy: 0.7221 - val_loss: 0.6090 - val_accuracy: 0.7378
Epoch 7/50
32/32 [==============================] - 0s 4ms/step - loss: 0.5656 - accuracy: 0.6666 - val_loss: 0.6111 - val_accuracy: 0.7191
Epoch 8/50
32/32 [

# Задание 3


Проверить качество модели не менее чем на трёх разных метриках. Вывести эти значения.

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predict = np.argmax(model1.predict(X_test), axis=-1)


In [ ]:
print(classification_report(y_test2,predict ))

              precision    recall  f1-score   support

           0       0.73      0.83      0.77       151
           1       0.73      0.59      0.65       116

    accuracy                           0.73       267
   macro avg       0.73      0.71      0.71       267
weighted avg       0.73      0.73      0.72       267

